# BENCHMARKING OF APPLICABILITY DOMAINS FOR MODELS PREDICTING PROPERTIES OF CHEMICAL REACTIONS

# Quantitative Reaction-Property model  predicting the rate concats of reactions

Nowadays, Quantitative Structure-Activity/Property Relationship (QSAR/QSPR) models are widely used for predicting properties of chemical compounds [[1], [2]]. The growing attention is attracted to  chemical reactions as objects of QSAR/QSPR-like modelling [[3],[4],[5]]. Below, you will find a model (**QRPR, Quantitative Reaction-Property model**) predicting the quantitative characteristic (rate constant) of bimolecular nucleophilic substitution reactions.

[1]: https://pubs.acs.org/doi/10.1021/jm4004285
[2]: https://www.sciencedirect.com/science/article/pii/B9780128015056000077?via%3Dihub
[3]: http://mr.crossref.org/iPage?doi=10.1070%2FRCR4746
[4]: https://pubs.acs.org/doi/abs/10.1021/acs.accounts.8b00087
[5]: https://onlinelibrary.wiley.com/doi/abs/10.1002/minf.201800104

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from os import environ
from CIMtools.preprocessing import Fragmentor, CGR, EquationTransformer, SolventVectorizer
from CIMtools.preprocessing.conditions_container import DictToConditions, ConditionsToDataFrame

def extract_meta(x):
    return [y[0].meta for y in x]

def x_generation(data_train, data_test):
    environ["PATH"]+=":/home/assima/env/bin"
    features = ColumnTransformer([('temp', EquationTransformer('1/x'), ['temperature']),
                                  ('solv', SolventVectorizer(), ['solvent.1']),
                                  ('amount', 'passthrough', ['solvent_amount.1']),])
    conditions = Pipeline([('meta', FunctionTransformer(extract_meta)),
                           ('cond', DictToConditions(solvents=('additive.1',), 
                                                     temperature='temperature', 
                                                     amounts=('amount.1',))),
                           ('desc', ConditionsToDataFrame()),
                           ('final', features)])
    graph = Pipeline([('CGR', CGR()), 
                      ('frg', Fragmentor(fragment_type=3, max_length=4, useformalcharge=True, version='2017')), 
                      ('scaler', StandardScaler())]) # All descriptors were normalized to zero mean and unit variance.

    pp = ColumnTransformer([('cond', conditions, [0]),
                            ('graph', graph, [0])])
    X_train = pp.fit_transform([[x] for x in data_train])
    X_test = pp.transform([[x] for x in data_test])
    return X_train, X_test

In [3]:
from tqdm import tqdm
from CGRtools import RDFRead

data = RDFRead('/home/assima/Assima_purple/home/Datasets/SN2_11_11_2019.rdf')
reactions = []
for n, r in tqdm(enumerate(data._data)):
    r.kekule() # leads to kekula formula for benzene rings
    r.implicify_hydrogens() # removes hydrogens
    r.thiele() # aromatizes benzene rings
    reactions.append(r)

del data

4830it [00:10, 442.36it/s]


In the study, we use **Random Forest Regression (RFR)** and **Gaussian Process Regression (GPR)** for building QRPR models

The number of trees in **RandomForestRegressor** is set to 500, while the only tuneable hyperparameter is the number of features selected upon tree branching (max_features). Other hyperparameters in **RandomForestRegressor** are set to default values. 

For **GPR** models, hyperparameters of noise level, alpha, and RBF kernel’s gamma values are adjusted. 

To obtain a reliable assessment of predictive performance and avoid overfitting, the nested cross-validation  procedure [[6](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-7-91)] is used. For each training/test split in the outer loop, the hyperparameters of **RandomForestRegressor** and **Gaussian Process Regression** models are tuned using grid search by minimizing the averaged RMSE of prediction (without AD application) estimated in the inner cross-validation loop on the outer training set, and the optimal models with the tuned hyperparameters are used to predict reaction properties on the outer test set. 

For simplicity, we will break on the first fold

In [4]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.utils import safe_indexing

kf = KFold(n_splits=5, random_state=1, shuffle=True)  

Y_true_rfr, Y_pred_rfr, Y_pred_gpr, Y_true_gpr = [], [], [], [] # collect all the values of logK

for train_index_ext, test_index_ext in kf.split(reactions):  # external set
    reactions_train = safe_indexing(reactions, train_index_ext)
    reactions_test = safe_indexing(reactions, test_index_ext)
    X_train, X_test = x_generation(reactions_train, reactions_test) # fragment descriptors
    Y_train = [float(x.meta['logK']) for x in reactions_train] # predictable property is the rate constants of the reactions
    Y_test = [float(x.meta['logK']) for x in reactions_test] 

    est = GridSearchCV(RandomForestRegressor(random_state=1, n_estimators=500),
                       {'max_features': [ None]},
                       cv=kf, verbose=1, scoring='neg_mean_squared_error', n_jobs=-1).fit(X_train, Y_train) # internal set 
    # [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 'auto', 'log2', None]
    Y_pred = est.predict(X_test)
    Y_pred_rfr.extend(Y_pred)
    Y_true_rfr.extend(Y_test)
    
    # All descriptors were normalized to zero mean and unit variance. 
    # In the case of GPR models, both descriptors and property values were normalized, 
    # because this provided better predictive performance
    
    scaler = StandardScaler()
    y_train_gpr = scaler.fit_transform(np.array(Y_train).reshape(-1, 1))
    y_test_gpr = scaler.transform(np.array(Y_test).reshape(-1, 1))
    Y_true_gpr.append(y_test_gpr)
    param_grid = {'kernel': [RBF(1e-6)], 'alpha': [1e-1]} 
    # [RBF(1e-6), RBF(1e-5), RBF(1e-4), RBF(1e-3), RBF(1e-2), RBF(1e-1), RBF(1), RBF(10), RBF(100), RBF(1000), RBF(10000)]
    gpr_grid = GridSearchCV(GaussianProcessRegressor(random_state=1), param_grid=param_grid, cv=kf,
                        scoring='neg_mean_squared_error', verbose=0, n_jobs=1).fit(X_train, y_train_gpr) # internal set
    Y_pred_GPR, Y_var = gpr_grid.best_estimator_.predict(X_test, return_std=True)
    Y_pred_gpr.append(Y_pred_GPR)
    break

/home/assima/env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   42.6s finished


# How we can assess the reliability of the model's predictions?

QSAR/QSPR models are not universal, and their predictive performance highly depends on similarity to training examples [[7](https://www.researchgate.net/publication/7498882_QSAR_applicabilty_domain_estimation_by_projection_of_the_training_set_descriptor_space_a_review), [8](https://www.researchgate.net/publication/7583123_Current_Status_of_Methods_for_Defining_the_Applicability_Domain_of_Quantitative_Structure-Activity_Relationships-The_Report_and_Recommendations_of_ECVAM_Workshop_52)]. Applicability Domain (AD) of a QSAR/QSPR model highlights a part of the chemical space containing those compounds for which the model is supposed to provide reliable predictions [[9](https://pubs.acs.org/doi/abs/10.1021/ci800151m), [10](https://pubs.acs.org/doi/10.1021/ci100253r)]. So, the problem of determining AD of a model is closely related to the problem of assessing the reliability of its predictions. According to the OECD (Organization of Economic Co-operation and Development) principles, QSAR/QSPR models should have “defined an applicability domain” [[11](https://www.oecd-ilibrary.org/environment/guidance-document-for-the-development-of-oecd-guidelines-for-testing-of-chemicals_9789264077928-en)]. 

Although numerous approaches are considered in the literature to assess the AD for the models predicting the properties of chemical compounds, ADs have almost never been applied to the models predicting characteristics of chemical reactions, and the problem of AD definition for chemical reactions has never been discussed in the literature. 

It is much more difficult to define AD for the models aimed at predicting different characteristics of chemical 
reactions in comparison with standard QSAR/QSPR models dealing with the properties of chemical compounds because it is necessary to consider several additional factors  (reaction representation, conditions, reaction type,  atom-to-atom mapping, etc) that are specific for chemical reactions and should be taken into account. 

Therefore, in this notebook, we will show the various AD definition methods extensively used in QSAR/QSPR studies, their modifications, as well as novel approaches designed by us for reactions.

## Approaches for defining applicability domain of QRPR model

AD definition approaches are considered as binary classifiers returning True for X-inliers (within AD) and False 
for X-outliers (outside AD). In this work, AD definition approaches are conditionally divided into two groups: 
(1) universal and (2) ML-dependent. For **Universal AD definition approaches**, only the Random Forest Regression (RFR) was used for building QRPR models (see above, *4 cell*). For **ML-dependent AD definition approaches**, 
both Random Forest Regression and Gaussian Process Regression machine learning methods were used for this 
purpose (sea below, ).

### Universal applicability domain definition approaches

Universal AD definition methods can be used on top of QRPR models, which can be implemented by any suitable machine learning method. For example, above, for implementing QRPR models we used the Random Forest Regressor. A part of them (**Bounding Box**, **Fragment Control**, **Reaction Type Control**) gives an answer whether a test object is within AD or not. When using **Bounding Box** and **Fragment Control** needn't choose a value for any adjustable parameter. When using **Reaction Type Control**, it is usually necessary to choose a value of adjustable hyperparameter. Such methods correspond to the applicability aspect of AD definition according to Hanser et al. [12](https://www.tandfonline.com/doi/full/10.1080/1062936X.2016.1250229).

For AD definition approaches which do not require hyperparameters selection, a regression QRPR model and an AD definition model were built on the external training set selected in each split of external cross-validation and both the property and applicability domain membership (within AD or out of AD) for the external test set were predicted. See below.

#### Fragment Control (FC)

In this case, if a Condensed Graph of Reaction representing a given reaction has fragments (subgraphs) missing in the training set, then it is considered to be an X-outlier (out of AD) whenever the corresponding QRPR model is applied. **Fragment Control** can formally be considered as a special case of **Bounding Box** for fragment descriptors. This method does not have adjustable parameters.

In [5]:
AD_FC = Pipeline([('cgr', CGR()), 
                            ('frg', Fragmentor(version='2017', 
                                               max_length=4, 
                                               useformalcharge=True,
                                               return_domain=True))]).fit(reactions_train).transform(reactions_test)

In [6]:
AD_FC.AD.values [:10]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

#### Bounding box

This approach defines AD as a D-dimensional hypercube with each edge spanning the range between the minimum and maximum values of the corresponding descriptor. If at least one descriptor for a given reaction is out of the range defined by the minimum and maximum values of the training set examples, the reaction is considered outside of the AD of the corresponding QRPR model. The method does not have adjustable hyperparameters.

In [7]:
from CIMtools.applicability_domain import Box

AD_BB = Box().fit(X_train).predict(X_test)

In [8]:
AD_BB[:10]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

#### Metrics

To build the best AD definition models, it is necessary to optimize their thresholds and (hyper)parameters
by maximizing some AD performance metric. We will use the following metrics:

(i) OIR (Out and in RMSE) - is the difference between RMSE of property prediction for reactions outside AD (denoted as RMSEOUT) and within AD (denoted as RMSEIN). The metric was first proposed by Sahigata et al [13](https://europepmc.org/article/pmc/pmc6268288). 

(ii) The Outliers Criterion metric shows how well AD definition detects Y-outliers. First, property prediction errors are estimated in cross-validation for all reactions in a dataset. The reactions for which the absolute prediction error is higher than 3×RMSE are identified as Y-outliers, while the rest are considered as Y-inliers. Y-Outliers (poorly predicted) that are predicted by AD definition as X-outliers (outside AD) are called true outliers (TO), while Y-inliers predicted by AD definition as X-inliers (within AD) are called true inliers (TI). False outliers (FO) are Y-inliers that are wrongly predicted by the AD definition as X-outliers, while false inliers (FI) are Y-outliers that are wrongly predicted by the AD definition as X-inliers. Then quality of outliers/inliers determination can be assessed using an analogue of the balanced accuracy and denoted as OD (Outliers Detection). 
OD = (TO/(TO+FI)+TI/(TI+FO))/2.

They are implemented in CIMtools.metrics.applicability_domain_metrics as well as slightly modified functions themselves are shown below. These functions will be needed later when working with 1-SVM

In [9]:
from sklearn.metrics import mean_squared_error, balanced_accuracy_score

def balanced_accuracy_score_with_ad(Y_pred_test, AD_pred):
    AD_true = abs(Y_pred_test[:, 0] - Y_pred_test[:, 1]) <= 3 * np.sqrt(mean_squared_error(Y_pred_test[:, 1],
                                                                                          Y_pred_test[:, 0]))
    return balanced_accuracy_score(AD_true, AD_pred)

In [10]:
def rmse_score_with_ad(Y_pred_test, AD_pred):
    AD_out_n = ~AD_pred
    s_n = sum(AD_pred)
    s_out_n = sum(AD_out_n)
    if s_n:
        RMSE_AD = np.sqrt((sum(map(lambda x: (((x[0] - x[1]) ** 2) * x[2]), zip(Y_pred_test[:, 0], Y_pred_test[:, 1], AD_pred)))) / s_n)
    else:
        RMSE_AD = 0

    if s_out_n:
        RMSE_AD_out_n = np.sqrt((sum(map(lambda x: (((x[0] - x[1]) ** 2) * x[2]), zip(Y_pred_test[:, 0], Y_pred_test[:, 1], AD_out_n)))) / s_out_n)
    else:
        RMSE_AD_out_n = 0    
    return RMSE_AD_out_n - RMSE_AD

#### The procedure for selecting the hyperparameters of QRPR and the AD definition models

Since hyperparameters of some AD definition methods need to be tuned, the above-mentioned nested cross-validation procedure was used, in which the inner 5-fold cross-validation loop was used for hyperparameter selection, whereas the outer 5-fold cross-validation loop was used for assessing predictive performance.
All hyperparameters of both QRPR and AD definition models were selected for each training/test split in the outer cross-validation loop by maximizing the OIR or OD metrics computed for the outer training set using the inner cross-validation loop. The selected hyperparameters were used to rebuild models on the outer training set, which were further used to predict reaction property and AD membership (within AD or out of AD) on the corresponding outer test set. After the completion of the outer loop, all values (predicted value and AD membership) predicted on individual outer test sets were merged, and the predictive performances of QRPR models with AD were assessed.

####  Reaction Type Control

If the reaction centre of a chemical reaction is absent in the reactions in the training set, it is considered out of AD (X-outlier). Reaction centre is detected using reaction signatures [14](https://pubs.acs.org/doi/10.1021/acs.jcim.9b00102). Signature creation includes (1) representation of a chemical reaction as a **Condensed Graph of Reaction**, (2) highlight one or more reaction centers which are identified as a set of adjacent dynamic atoms and bonds on the CGR and (3) environment atoms of a certain radius R for each of the reaction centers, (4) introducing canonical numbering of atoms of the reaction center with the environment using an algorithm similar to the Morgan algorithm, (5) the signature is encoded by SMILES-like canonical string generated by CGRtools library [14](https://pubs.acs.org/doi/10.1021/acs.jcim.9b00102). For every atom hybridization and element label, as well as bond order is encoded in a signature. Since the method does not consider the whole structure, but only its reaction center with its closest environment, in order to be able to distinguish whether the aromatic cycle is part of the reaction center or its closest substituent, we introduced a separate type of hybridization for aromatic carbon atoms. We also used sp3, sp2, sp hybridization to describe the hybridization of not aromatic atoms. Hence, the signature includes information both on the reaction centre itself and its closest environment of radius R. The radius of environment included into the signature is a hyperparameter of the method. If the environment is set to 0, the reaction signature includes only atoms of the reaction centre.

In [11]:
from CIMtools.model_selection import rtc_env_selection
from CIMtools.applicability_domain import ReactionTypeControl

# First find the optimal radius. For this reason, we need to use rtc_env_selection function.
# We must pass variables such as X_train, Y_train , best_model - to build a model, reactions_train - list of reactions,
# envs - list of radius values and set the metric by which the hyperparameter will be optimized

score = 'ba_ad' # or 'rmse_ad'
best_r = rtc_env_selection(X=X_train, y=Y_train, data=reactions_train, envs=[0, 1], 
                           reg_model=est.best_estimator_, 
                           score=score) # as you can see we use only the training set for optimize their thresholds and (hyper)parameters
# [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
AD_RTC_cv = ReactionTypeControl(env=best_r).fit(reactions_train).predict(reactions_test)

/home/assima/env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/assima/env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/assima/env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/assima/env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(m

In [12]:
best_r

1

In [13]:
AD_RTC_cv[:10]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

Another part of them Universal AD definition methods (**Leverage**, **Nearest Neighbors approach**, **One-Class SVM**, **Two-Class X-inlier/Y-outlier classifier**) return a continuous value indicating the reliability of prediction. When using these methods, it is usually necessary to choose a threshold for such a value, and for some of them, the values of adjustable hyperparameters. Such methods correspond to the reliability aspect of AD definition according to Hanser et al. [12](https://www.tandfonline.com/doi/full/10.1080/1062936X.2016.1250229)

#### Leverage

This method is based on the Mahalanobis distance to the centre of the training-set distribution. The leverage h of a chemical reaction is calculated based on the “hat” matrix as h=(xiT(XTX)-1xi), where X is the training-set descriptor matrix, and xi is the descriptor vector for the reaction i. The leverage threshold is usually defined as 
h*=3*(M+1)/N, where M is the number of descriptors and N is the number of training examples. Chemical reactions with leverage values h > h* are considered to be chemically different from the training-set reactions, so they are marked as X-outliers. This approach is denoted hereafter as Leverage. 

In [14]:
from CIMtools.applicability_domain import Leverage

AD_Leverage = Leverage(threshold='auto').fit(X_train, Y_train).predict(X_test)

In [15]:
AD_Leverage[:10]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [16]:
# if you want to predict the distances for X to center of the training set, you should write 
Leverage(threshold='auto').fit(X_train, Y_train).predict_proba(X_test) 

array([4.15652350e-02, 1.98145197e-02, 1.46696292e-02, 1.48282359e-02,
       1.52431107e-02, 1.23464406e-01, 5.20473763e-02, 5.74329729e-02,
       4.20180764e-02, 4.20079504e-02, 2.30468541e-02, 2.18049682e-02,
       2.86753369e-02, 2.44993317e-02, 5.09700774e-02, 9.61464732e-02,
       2.27451111e-01, 1.63327780e-02, 7.92303786e-02, 1.16420112e-01,
       4.58887340e-02, 3.59907720e-02, 4.07309659e-02, 4.43190571e-02,
       2.61784426e-01, 2.13167612e-01, 2.04666722e-01, 3.25919047e-02,
       1.16788774e-01, 1.17492578e-01, 1.16813131e-01, 8.22376478e-02,
       1.15253605e-02, 3.38165164e-02, 1.02515342e-01, 7.41550829e-02,
       1.53142791e-01, 5.01867128e-02, 5.39678368e-01, 7.58144403e-02,
       2.16744657e-01, 2.40962855e+09, 4.92783114e-02, 4.98929354e-02,
       2.36371597e-02, 2.37591924e-02, 3.35730464e-02, 3.39811253e-02,
       1.47671365e-02, 3.33357480e-01, 2.61120053e-02, 2.61477126e-02,
       8.88706593e-02, 1.14265996e-02, 1.21605609e-02, 1.21389393e-02,
      

#### Lev_cv

The drawback of it is the absence of strict rules for choosing the threshold h*. As an alternative, an optimal threshold value h* can be found using an internal cross-validation procedure by maximizing some AD performance metrics.

In [17]:
AD_Lev_cv = Leverage(threshold='cv', score=score, 
                     reg_model=est.best_estimator_).fit(X_train, Y_train).predict(X_test)

/home/assima/env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/assima/env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/assima/env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/assima/env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(m

In [18]:
AD_Lev_cv[:10]

array([ True,  True,  True,  True,  True, False,  True,  True,  True,
        True])

#### Z-1NN

This AD definition is based on the distance(s) between a current reaction and the closest training-set reaction(s). Usually, one nearest neighbour is considered (k=1). If the distance is not within the user-defined threshold, then the prediction is considered unreliable and the reaction is considered as X-outlier. The threshold value is commonly taken as Dc=Zσ+<y>, where <y> is the average and σ is the standard deviation of the Euclidean distances between nearest neighbours in the training set, Z is an empirical parameter to control the significance level, with the recommended value of 0.5.

In [19]:
from CIMtools.applicability_domain import SimilarityDistance

AD_Z1NN = SimilarityDistance(threshold='auto').fit(X_train, Y_train).predict(X_test)

In [20]:
AD_Z1NN[:10]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [21]:
# Returns the value of the nearest neighbor from the training set.
SimilarityDistance(threshold='auto').fit(X_train, Y_train).predict_proba(X_test)

array([1.02001345e-04, 5.53194200e-05, 2.90006968e-01, 1.60658619e-04,
       3.06629569e-04, 2.35221173e-01, 3.33928136e-01, 5.89367610e-05,
       1.60658619e-04, 1.45970950e-04, 4.15819323e-04, 1.68378995e-04,
       9.02001109e-01, 1.10650395e+00, 1.35528595e-01, 0.00000000e+00,
       9.68561444e+00, 0.00000000e+00, 8.08610280e-05, 6.21910250e+00,
       1.07160581e+00, 2.49572261e+00, 3.96237352e-01, 0.00000000e+00,
       0.00000000e+00, 1.92111473e+00, 1.92111473e+00, 4.86333198e-05,
       1.12532006e-06, 9.50442516e-05, 4.37599359e-05, 9.28872322e-05,
       0.00000000e+00, 4.56781536e-01, 3.31064411e+00, 1.05339199e-04,
       9.62943201e-08, 0.00000000e+00, 2.88661719e+00, 0.00000000e+00,
       0.00000000e+00, 5.75867802e+00, 9.88196756e-05, 8.25195377e-05,
       4.71512744e-05, 5.01863557e-05, 9.28872322e-05, 3.38049547e-06,
       5.91917911e-05, 3.14835139e-05, 3.36203000e-05, 3.27651480e-05,
       1.12525909e-04, 9.88196756e-05, 9.88196756e-05, 2.04158874e-04,
      

#### Z-1NN_cv

An optimal threshold can be found using an internal cross-validation procedure by maximizing some AD performance metrics.

In [22]:
AD_Z1NN_cv = SimilarityDistance(score=score, threshold='cv', 
                                reg_model=est.best_estimator_).fit(X_train, Y_train).predict(X_test)

/home/assima/env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/assima/env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/assima/env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/assima/env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(m

In [24]:
AD_Z1NN_cv[:10]

array([ True,  True, False,  True, False, False, False,  True,  True,
        True])

#### Two-Class X-inlier/Y-outlier Classifier

In this case, a binary classifier learns to distinguish Y-inliers from Y-outliers. First, QRPR models are built to predict quantitative characteristics of chemical reactions. Chemical reactions with higher prediction error estimated in cross-validation (more than 3×RMSE) are labelled as Y-outliers, while the remaining reactions are labelled as Y-inliers. After that, a binary classification model is trained to discriminate between them and provide a confidence score that a given reaction is a Y-inlier for the corresponding QRPR model. Although this method seems quite straightforward, we have not found its application in literature. Unfortunately, this method cannot be applied if there are no or too few Y-outliers. In this study, Random Forest Classifier implemented in scikit-learn library was used for building the binary classification model. The method requires setting the values of two hyperparameters: max_features (the values of features selected upon tree branching) and probability threshold P. If the predicted probability of belonging to the X-inliers is greater than P, the prediction of reaction characteristics by the QRPR model for it is considered reliable (within AD). Other hyperparameters of the Random Forest Classifier was set to defaults, except the number of decision trees in Random Forest Classifier which was set to 500.

In [25]:
import numpy as np
from sklearn.model_selection import cross_val_predict
from CIMtools.applicability_domain import TwoClassClassifiers
from sklearn.ensemble import RandomForestClassifier

Y_predicted = cross_val_predict(RandomForestRegressor(random_state=1, n_estimators=500, 
                                                      max_features=est.best_params_['max_features']), 
                                X_train, Y_train, cv=kf, verbose=1, n_jobs=1)
Y_pr_ts = np.column_stack((Y_predicted, Y_train))

Y_R_int = abs(Y_pr_ts[:, 0] - Y_pr_ts[:, 1]) <= 3 * np.sqrt(mean_squared_error(Y_pr_ts[:, 1], Y_pr_ts[:, 0]))
best_model_clf = GridSearchCV(RandomForestClassifier(n_estimators=500, random_state=1),
                      {'max_features': [None]},
                      cv=kf, verbose=1, n_jobs=1).fit(X_train, Y_R_int) 
# [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 'auto', 'log2', 'sqrt', None]
AD_2CC = TwoClassClassifiers(threshold='cv', score=score, reg_model=est.best_estimator_, 
                             clf_model=best_model_clf.best_estimator_).fit(X_train, Y_train).predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  2.5min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  4.6min finished
/home/assima/env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/assima/env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/assima/env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/assima/env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated 

In [26]:
AD_2CC[:10]

array([ True,  True,  True,  True,  True,  True, False,  True,  True,
        True])

### 1-SVM

The one-class Support Vector Method reveals highly populated zones in descriptor space by maximizing the distance between a separating hyperplane and the origin in the feature space implicitly defined by some Mercers’ kernel. The decision function of such model returns (+1) for the reactions which fall into highly populated zones (within AD, i.e. X-inliers) and (−1) - for the reactions outside of AD (X-outliers). 1-SVM models were built in this study using the scikit-learn library. The method requires the fitting of two hyperparameters: nu (which defines the upper bound percentage of errors and lower bound percentage of support vectors) and gamma (parameter of RBF kernel which is used), the optimal values of which can be found in cross-validation

In [27]:
from sklearn.metrics import make_scorer
from sklearn.svm import OneClassSVM

if score == 'ba_ad':
    scorer_for_svm = make_scorer(balanced_accuracy_score_with_ad, greater_is_better=True)
else:
    scorer_for_svm = make_scorer(rmse_score_with_ad, greater_is_better=True)
AD_SVM = GridSearchCV(OneClassSVM(), {'nu': [0.001, 0.005],
                                       'gamma': [1e-6]},
                       cv=kf, verbose=1, scoring=scorer_for_svm, n_jobs=-1).fit(X_train, Y_pr_ts).predict(X_test)
# [0.001, 0.005, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
# [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1000, 10000]

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:    4.1s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    4.2s finished


In [28]:
AD_SVM[:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

## ML-dependent applicability domain definition approaches

### RFR_VAR

The variance of predictions made by an ensemble of QSAR/QSPR models is often used as a score for determining their AD. In this study, we extend this to QRPR models. If individual models trained on randomly selected subsets of the initial training set predict close property values than the averaged prediction made by their ensemble is considered reliable. Here, we consider a chemical reaction to be within AD (inlier) if the variance of property values predicted by the ensemble of models is less than a given threshold σ. The optimal value for σ can be found using internal cross-validation procedure by maximizing an AD quality metrics (see below). One of the approaches to estimate the prediction variance needed for this purpose is to build a QRPR model on the whole training set using the Random Forest Regression (RFR) machine learning method, which provides the mean (which is considered as a predicted value of the reaction property) and the variance of predictions (which is considered as a measure of prediction confidence) made by individual Random Trees individual models. This approach is denoted hereafter as RFR_VAR. In this study, a modified version of the Random Forest Regression method (RFR, 500 trees) implemented in scikit-learn library was used

In [38]:
from sklearn.base import clone
from numpy import hstack, unique

from CIMtools.metrics.applicability_domain_metrics import balanced_accuracy_score_with_ad, rmse_score_with_ad

def threshold(X, y, model, score):
    cv = KFold(n_splits=5, random_state=1, shuffle=True)
    model_int = clone(model)

    threshold_value, score_value = 0, 0
    Y_pred, Y_true, AD = [], [], []
    for train_index, test_index in cv.split(X):
        x_train = safe_indexing(X, train_index)
        x_test = safe_indexing(X, test_index)
        y_train = safe_indexing(y, train_index)
        y_test = safe_indexing(y, test_index)
        model_int.fit(x_train, y_train)

        AD.extend(model_int.predict_proba(x_test)) 
        Y_pred.append(model_int.predict(x_test))
        Y_true.append(y_test)
    AD_stack = hstack(AD)
    AD_ = unique(AD_stack)
    for z in AD_:
        AD_new = AD_stack <= z
        if score == 'ba_ad':
            val = balanced_accuracy_score_with_ad(Y_true=hstack(Y_true), Y_pred=hstack(Y_pred), AD=AD_new)
        elif score == 'rmse_ad':
            val = rmse_score_with_ad(Y_true=hstack(Y_true), Y_pred=hstack(Y_pred), AD=AD_new)
        if val >= score_value:
            score_value = val
            threshold_value = z
    return threshold_value, score_value

In [31]:
from random_forest_variance import RandomForestRegressor2

/home/assima/env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/assima/env/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
/home/assima/env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecate

In [39]:
est_var = RandomForestRegressor2(random_state=1, n_estimators=500,
                                 max_features=est.best_params_['max_features']).fit(X_train, Y_train)
AD_est_var_values = est_var.predict_proba(X_test)
min_h_param_RFR_VAR = threshold(X=X_train, y=Y_train, model=est_var, score=score) # для нахождения отсечки
AD_RFR_VAR = AD_est_var_values <= min_h_param_RFR_VAR[0]


/home/assima/env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/assima/env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/assima/env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/assima/env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(m

In [40]:
AD_RFR_VAR[:10]

array([ True, False,  True,  True,  True,  True,  True,  True,  True,
        True])

### GPR-AD

**Gaussian Process Regression (GPR)** assumes that the joint distribution of a real-valued property of chemical reactions and their descriptors is multivariate normal (Gaussian) with the elements of its covariance matrix computed by means of special covariance functions (kernels). For every reaction, a GPR model produces using the Bayes’ theorem a posterior conditional distribution (so-called prediction density) of the reaction property given the vector of reaction descriptors. The prediction density has normal (Gaussian) distribution with the mean corresponding to predicted value of the property and the variance corresponding to prediction confidence. If the variance is greater than a predefined threshold σ, the chemical reaction is considered as X-outlier (out of AD). This AD definition method is denoted as GPR-AD. The method requires adjustment of three hyperparameter - alpha which stands for the noise level (also acts as regularization of the model), the parameter gamma of the RBF kernel which represents the covariance function and variance threshold σ. The optimal values of hyperparameters are determined using internal cross-validation. Other hyperparameters of Gaussian Processes are set by default.

In [42]:
from CIMtools.applicability_domain import GPR_AD
AD_GPR = GPR_AD(threshold='cv', score='ba_ad', 
                gpr_model=gpr_grid.best_estimator_).fit(X_train, Y_train).predict(X_test)

/home/assima/env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/assima/env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/assima/env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/assima/env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(m

In [43]:
AD_GPR[:10]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

## Zero Models and "Perfect Model"

Zero models can be suggested to compare different AD definitions with some simple rules. Zero models include “optimistic” AD definitions assuming that all reactions are within AD (denoted as OZ), while the “pessimistic” AD definition assumes that all reactions are out of AD (denoted as PZ). In addition to zero models, the “perfect” AD definition can be proposed for comparison. The “Perfect AD model” assumes that all X-inliers are Y-inliers (i.e. for all reactions within AD absolute property prediction error is lower than 3×RMSE), and all X-outliers are Y-outliers (i.e. for all reactions outside AD absolute property prediction error is higher than 3×RMSE).

In [44]:
AD_ZeroModel_One = np.ones(X_test.shape[0])  # all reactions are in AD
AD_ZeroModel_Zero = np.zeros(X_test.shape[0])  # all reactions are not AD
AD_Perfect = abs(Y_pred - Y_test) <= 3 * np.sqrt(-est.best_score_)

And so we go through the remaining 4 folds. 
Then we collect all the results and calculate the characteristics...

In [ ]:
# def assembly_of_results(name, AD_list, num, AD, RTC1=None):
#     AD_list[num].extend(AD)
#     if RTC1 is not None:
#         AD_list[num+1].extend(np.logical_and(AD, RTC1))
#     print('{}_is_done!'.format(name))

In [ ]:
# from CIMtools.applicability_domain import ReactionTypeControl
# RTC_1 = ReactionTypeControl(env=1).fit(reactions_train).predict(reactions_test)
# assembly_of_results('Reaction Type Control with R=1', AD_rfr, 0, RTC_1)